In [ ]:
from datascience import *
from prob140 import *
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
%matplotlib inline
import numpy as np
from scipy import stats

from client.api.assignment import load_assignment
autograder = load_assignment('main.ok')

# Lab 11: MLE, Chi-Squared, and Goodness of Fit #
Suppose your data are drawn from a population in which there are $c$ categories of individuals. Suppose that in the population, the proportion of individuals in Category $i$ is $p_i$, so that $\sum_{i=1}^c p_i = 1$.

Suppose your model is that your data are like the results of random draws made with replacement from the population. How well does the model fit the data?

One way in which data scientists answer this question is by testing for the rather awkwardly named "goodness of fit". 

In class you studied the test statistic
$$
\chi^2 ~ = ~ \sum_{i=1}^c \frac{(O_i - E_i)^2}{E_i}
$$

where $O_i$ is the observed count in category $i$ and $E_i$ is the count expected under the model.

If the sample size is large, the distribution of this statistic is approximately chi-squared, with degrees of freedom that depend on the number of categories and on whether parameters are being estimated in the calculation of the expected counts. 

In class you saw some reasons why this is true, and a proof in the case $c = 2$. In this lab you will examine the chi-squared statistic by simulation and use it to test the goodness of fit of different models.

## Part 1. Preliminaries ###
There's going to be a lot of plotting in this lab, all of it involving chi-squared density curves and simulated chi-squared statistics. In this part of the lab you will define some functions to do this.

A useful assortment of code details:
- For a positive integer `n`, the option `bins=n` used with the Table method `hist` results in a histogram with `n` equal bins.
- To create an array of `n` simulated values of a chi-squared random variable with `d` degrees of freedom, use `stats.chi2.rvs(d, size=n)`.
- To create an array of the values of the chi-squared (d) density evaluated at the points in the array `x`, use `stats.chi2.pdf(x, d)`.
- In this notebook, when two line plots are drawn on the same axes, the first plot drawn is blue and the second is red.

### a) ###
Define a function `array_hist` that takes as its arguments a numerical array and a number of bins; the function displays a histogram of the array with the specified number of equal-sized bins. In the definition of such a function there is no need for a `return` statement. Your definition should consist of just one line of code that results in the displayed histogram.

In [ ]:
def array_hist(array, num_bins):
    ...

### b) ###
Use `array_hist` to draw the empirical histogram of 5000 simulated values from the chi-squared (5) distribution, that is, the chi-squared distribution with 5 degrees of freedom. Use a number of bins that you think creates a good display of the shape of the distribution. You shouldn't need more than two lines of code.

What are the expected value and SD of the chi-squared (5) distribution?

In [ ]:
ev = ...
sd = ...
ev, sd

Are your answers above consistent with the mean and SD of the simulated values?

In [ ]:
... , ...

*Provide your answer and reasoning in this Markdown cell.*

### c) ###
Plotting density curves involves making decisions about the interval over which to plot the curve. Since all "curves" drawn in a computing environment consist of segments of straight lines, you also have to decide how many points you are going to plot so that "joining the dots" yields a graph that looks like a curve and not just joined-up straight line segments.

Define a function `plot_chi2` that takes three arguments:
- left end of plotting interval
- right end of plotting interval
- degrees of the freedom of the chi-squared distribution

The function should display a plot of the density of the chi-squared, with values plotted at roughly 100 equally spaced points (99 or 101 is fine). Use as many lines of code as you need. 

In [ ]:
def plot_chi2(left, right, df):
    step = ...
    x = np.arange(..., ..., step)
    plt.plot(x, ...)

Use `plot_chi2` to plot the chi-squared (5) density over the interval (0, 25).

### d) ### 
Use `array_hist` and `plot_chi2` to overlay a graph of the chi-squared (5) density over 5000 simulated values from that distribution. You are welcome to use the values from Part (b) if you saved those. Make sure you plot the density over the entire range of your simulated values.

In [ ]:
...
...

### e) ###
Define a function `chi2_statistic` that takes two arrays of counts (the second array being the expected counts under the model) and returns the chi-squared statistic.

In [ ]:
def chi2_statistic(array_obs, array_exp):
    ...

## Part 2. ACLU Alameda County Jury Selection Study ###

Recall the Data 8 example on [jury selection](https://www.inferentialthinking.com/chapters/10/1/jury-selection.html) in Alameda county. Skim the section if you need to. The data are given below.
- `panel_counts` consists of the number of jury panelists in each of the categories Asian, Black, Hispanic, White, and Other, in that order.
- `eligible_proportions` consists of the proportions of eligible jury panelists in the corresponding categories.

In [ ]:
panel_counts = make_array(384, 117, 114, 780, 58)
eligible_proportions = make_array(0.15, 0.18, 0.12, 0.54, 0.01)

### a) ###
Find the sample size and an array of the expected counts in all the categories under the model of random draws from the eligible population. Remember that expected counts don't have to be integers.

In [ ]:
sample_size = ...
sample_size

In [ ]:
model_exp_counts = ...
model_exp_counts

### b) ###
Calculate the observed value of the chi-squared statistic. Under the model, what is the expected value of the statistic? See the Course Notes if you have forgotten.

At this point it should be abundantly clear that the model of random draws doesn't fit the data. If you insist on finding $P$-values, you would have to find the appropriate right hand tail probability. Find it, and remember that all the `stats` function calls for the different distributions have a parallel structure: `stats.chi2.cdf(x, df)` gives the cdf of the chi-squared (`df`) distribution at `x`.

In [ ]:
...

So Python doesn't think the observed statistic is anywhere near the chi-squared distribution that should hold under the model. For comparison, find the critical value of the statistic for deciding between whether the model holds or not, using 5% as the cutoff for the $P$-value. You'll need to use the analog of `stats.norm.ppf(.8, 0, 1)` which gives the 80% point of the standard normal curve.

In [ ]:
...

### c) ###
Parts (a) and (b) show how if you believe the statistic has the appropriate chi-squared distribution under the model, no simulation or bootstrap is needed for finding the $P$-value. But we never actually proved that the statistic is approximately chi-squared when the number of categories is larger than 2. In this part you will confirm the distribution by a simulation study.

The function `np.random.multinomial` takes a sample_size and a categorical distribution as its arguments, and returns an array consisting of the counts in the various categories based on a random sample drawn with replacement from the distribution. That is, it provides the multinomial counts in the different categories.

Use `np.random.multinomial` to get the counts in a random sample from the eligible jury panelist population; the sample size should be the same as that in the ACLU study. You calculated it earlier.

In [ ]:
simulated_counts = np.random.multinomial(... , ...)

Use `chi2_statistic` to calculate a chi-squared statistic based on `simulated_counts` and `model_exp_counts`, and check that it is considerably smaller than the chi-squared statistic you got earlier based on the panelists.

In [ ]:
chi2_statistic(... , ...)

### d) ###
Under the model of random draws, simulate the chi-squared statistic 10,000 times. The simulated values should be collected in the array `chi2_vals`. Please don't display the 10,000 values.

In [ ]:
chi2_vals = make_array()
for i in ...:
    ...

### e) ###
Draw the histogram of the simulated statistics in `chi2_vals` and overlay the chi-squared density you used in part (b). You might have to experiment with the number of bins to get a good display.

In [ ]:
...
...

In part (b) you found the critical value of this distribution assuming the 5% cutoff for $P$-values. What proportion of the simulated values in `chi2_vals` are at least as large as this cutoff?

#### Summary of Part 2 ####
In parts (a) and (b) you assumed that under the model of random draws the statistic has the distribution stated in class, and you tested whether the model fits the data. 

In parts (c) through (e) you performed a simulation study to confirm that under the model of random draws the statistic does have the distribution stated in class.

## Part 3. Simulating $\chi^2$: Binomial $(5, 1/2)$ ###
Now we move from sampling from a finite (though large) population of individuals to data in an i.i.d. sequence $X_1, X_2, \ldots , X_n$. This a slightly more abstract version of sampling with replacement from a finite population.

This part of the lab is a simulation study to confirm the following:

If there are $c$ categories in the underlying distribution of the $X$'s and the proportions in all $c$ categories are known, then the chi-squared statistic has the chi-squared distribution with $c-1$ degrees of freedom.

We are going to check this in the case where $X_1, X_2, \ldots , X_n$ are i.i.d. binomial (5, 1/2).

### a) ###
Set the sample size equal to 500 and simulate an i.i.d. sample of 500 binomial (5, 1/2) variables.

In [ ]:
sample_size = 500
binom_data = stats.binom....

### b) ###
The function `sample_counts` takes as its arguments a non-negative integer-valued data array and an integer `n`, and returns an array of the counts of the integers 0 through `n` in the data array. 

Notice the strange `n+2` in the range for the bins. That's because of an endpoint convention: for the Table method `bin`, intervals include the left endpoint and not the right, so we have to be careful about the last bin. You don't have to worry about this. The function takes care of it.

Run the cell to define the function.

In [ ]:
def sample_counts(data, n):
    data_tbl = Table().with_column('Sample', data)
    obs_counts = data_tbl.bin(bins=np.arange(n+2))
    return obs_counts.column(1)[0:n+1]

Construct an array of the counts of `binom_data` in the categories 0, 1, 2, 3, 4, 5.

In [ ]:
obs_counts = sample_counts(... , ...)
obs_counts

Construct an array of the expected counts under the model that the data come from the binomial (5, 1/2) distribution. 

In [ ]:
model_exp_counts = ...
model_exp_counts

### c) ###
The two arrays above should resemble each other, apart from chance variation, since the `sample_counts` array was created by sampling at random from the binomial distribution. To quantify how close, find the chi-squared statistic.

In [ ]:
chi2_statistic(..., ...)

Now simulate the chi-squared statistic 10,000 times just as you simulated one value of the statistic above. Overlay the appropriate chi-squared curve over the histogram of the simulated values. The result from class implies that the two should be fairly close. 

### Part 4. Simulating $\chi^2$: Binomial $(5, p)$ ###
Now we're going to assume $X_1, X_2, \ldots , X_n$ is i.i.d. from a binomial distribution with $n = 5$ and unknown $p$. Of course we will actually know $p$ because we're running a simulation study. But when we calculate the chi-squared statistic we are going to pretend that we don't know $p$. The goal is to see how this affects the distribution of the statistic.

### a) ###
Generate a sample of 10 i.i.d observations from the binomial (5, 1/2) distribution, and promptly forget that you know the value of $p$. Based on these 10 observations, how can you find the MLE of $p$? No calculus is necessary. Remember the related result from class, and see how you can use it:
- In total, how many Bernoulli trials were included to get the data in your sample?
- How can you figure out the overall proportion of successes in all of those trials?

There are a couple of different reasonable ways to get at this. Pick your favorite one, and calculate the MLE of $p$.

*Provide your answer and reasoning in this Markdown cell.*

In [ ]:
data = stats.binom.rvs(...)
p_hat = ...
p_hat

### b) ### 
Get an array of the observed counts in all the categories, and the corresponding expected counts under the model of random draws from binomial $(5, \hat{p})$.

### c) ###
Now simulate a chi-squared statistic 10,000 times. Each time:
- Generate an i.i.d. sample of 500 binomial (5, 1/2) random variables.
- Calculate the MLE of $p$ based on the sample.
- Construct an array of observed counts in the different categories in the sample.
- Construct an array of expected counts under the model of random draws from binomial $(5, \hat{p})$.
- Calculate the ch-squared statistic.
- Repeat, starting again at the top of this list.

Once you have the 10,000 simulated values of the statistic, draw their empirical histogram using `array_hist` and overlay two chi-squared curves over it:
- chi-squared (5)
- chi-squared (4)

### c) ###
Remember that `plot` uses blue for the first plot in the cell and red for the second. If you approximate your empirical distribution by a chi-squared distribution, how many degrees of freedom should you use?

*Provide your answer and reasoning in this Markdown cell.*

### Part 5. Fitting a Poisson Distribution ###
Let $X_1, X_2, \ldots, X_n$ be i.i.d. Poisson $(\mu)$ random variables. 

### a) ###
Find the maximum likelihood estimate of $\mu$.

*Provide your answer and reasoning in this Markdown cell.*

### b) ### 
The table `kicks` contains a classic data set presented by [Ladislaus Bortkiewicz](https://en.wikipedia.org/wiki/Ladislaus_Bortkiewicz) (1868-1931) on deaths by horse kicks in the Prussian army. The Poisson distribution happens to fit this grim little data set rather well, and was central to Bortkiewicz's book *The Law of Small Numbers*. 

We got the data from a University of Utah Math Department [website](http://www.math.uah.edu/stat/data/HorseKicks.html). Run the cell below to see it.

In [ ]:
kicks = Table.read_table("HorseKicks.csv")
kicks.show()

Each cell represents the number of deaths by horse kicks in a particular cavalry corps in a particular year. Each row corresponds to a year and each column corresponds to one cavalry corps. There are 14 corps (the column labels C12 and C13 are missing) and 20 years, hence 280 observations in all. 

The model is that the 280 values are observations on i.i.d. Poisson random variables. Our goal is to figure out which Poisson, if any.

Complete the cell below to construct one array `data` consisting of all 280 values, and check that your array has the right number of values.

In [ ]:
kicks = kicks.drop('Year')

data = make_array()
for column in kicks.columns:
    data = np.append(... , ...)

len(data)

### c) ### 
For integer valued data, the `prob140` method `Plot(emp_dist(array_name))` can be used to get a histogram of the values in `array_name`. Use it to display the histogram of `data`.

In [ ]:
...

The maximum of the sample is 4. Happily, not many of the cavalry were dying of horse kicks in any given year. This "rare event" scenario is where the Poisson distribution can be a good model to use.

Construct an array of the sample counts in each of the categories 0 through 4.

### d) ###
Under the model that the data are i.i.d. Poisson $(\mu)$, find the numerical value of the MLE of $\mu$.

In [ ]:
mu_hat = ...
mu_hat

To find the expected proportions under the model, note that as the Poisson is a distribution on infinitely many values, we are going to have to sweep the tail into one maximum value. Our categories will be 0, 1, 2, and "3 or more".

Why not "4 or more"? That's because "4 or more" has a very small probability if $\mu$ is around 0.7. The chi-squared test should not be used if there are rare categories.

Construct an array `exp_proportions` that contains the Poisson (`mu_hat`) probabilities of Categories 0, 1, 2, and "3 or more", and check that it sums to 1.

In [ ]:
...

Find the expected counts in the four categories.

In [ ]:
exp_counts = ...
exp_counts

### e) ###
Take another look at the observed counts, and collapse the last two cells into one. This is to match the categories in the array of expected counts.

In [ ]:
obs_counts

In [ ]:
obs_counts = ...
obs_counts

Without doing any further calculation, say whether you think the Poisson model fits.

*Provide your answer and reasoning in this Markdown cell.*

### f)

Calculate the chi-squared statistic.

### g)

If the model holds, the test statistic should be compared with a particular chi-squared distribution. Which one, and why?

*Provide your answer and reasoning in this Markdown cell.*

What is the expected value of the chi-squared distribution you have specified? Based on that value, do you think the model fits the data?

*Provide your answer and reasoning in this Markdown cell.*

Run the cell below to see overlaid histograms of the data and the Poisson fit.

In [ ]:
x = np.arange(4)
poisson = Table().values(x).probability(exp_proportions)
Plots('Sample Counts', emp_dist(data), 'Poisson Fit', poisson)

When the evidence for whether or not the model fits is as clear as in the jury selection example and in this one about horse kicks, it's really not necessary to find a $P$-value.

In [ ]:
_ = autograder.grade('q1')

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [autograder.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

In [ ]:
import gsExport
gsExport.generateSubmission()